In [1]:
import json
import spotipy
import pandas as pd
import numpy as np
import time
from requests.exceptions import ReadTimeout
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
with open('config/config.json', 'r') as f:
    config = json.load(f)
CLIENT_ID = config['CLIENT_ID']
CLIENT_SECRET = config['CLIENT_SECRET']

In [3]:
# Authentication - without user
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager, requests_timeout = 10, retries = 10)

# Functions

In [4]:
def get_playlist_tracks(playlist_URI):
    results = sp.playlist_tracks(playlist_URI)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [5]:
def return_unique_tracks_with_data(tracks):
    track_names = []
    tracks_with_data = []
    for cnt, track in enumerate(tracks):
        # We check for track data + name
        if track['track'] is None:
            continue
        if track['track']['name'] is None:
            continue

        track_names.append(track['track']['name'])
        tracks_with_data.append(track)
    
    tracks_with_data = np.array(tracks_with_data)
    track_names = np.array(track_names)
    
    #track_names = np.array([track["track"]["name"] for track in tracks if track['track'] is not None or track['track']['name'] is not None])
    unique_track_names, idx = np.unique(track_names, return_index = True) 
    
    unique_tracks = tracks_with_data[idx]
    unique_track_names = track_names[idx]

    return unique_tracks, unique_track_names

In [6]:
def get_multiple_playlist_tracks(playlist_links):
    track_list = []
    for playlist_link in playlist_links:
        playlist_URI = playlist_link.split("/")[-1].split("?")[0]
        
        tracks = get_playlist_tracks(playlist_URI)
        
        track_list = track_list + tracks
        
    return np.array(track_list)

In [7]:
def get_audio_features_from_tracks(sp, tracks):    
    # We can loop over all tracks and get information about them
    keys = ["artist_name", "artist_pop", "track_name", "track_pop", "album", "key", "mode", "tempo", "duration_ms", "time_signature", "energy", "danceability", "instrumentalness"]
    artist_keys = ["artist_name", "artist_pop", "track_name", "track_pop", "album"]
    audio_keys = ["key", "mode", "tempo", "duration_ms", "time_signature", "energy", "danceability", "instrumentalness"]
    dict_data = {key: np.array([]) for key in keys}
    
    for cnt, track in enumerate(tracks):
        print(f'track {cnt} out of {len(tracks)-1}')
        
        try:
            #URI
            track_uri = track["track"]["uri"]

            #Track name
            track_name = track["track"]["name"]

            #Main Artist
            artist_uri = track["track"]["artists"][0]["uri"]
            artist_info = sp.artist(artist_uri)

            #Name, popularity, genre
            artist_name = track["track"]["artists"][0]["name"]
            artist_pop = artist_info["popularity"]
            artist_genres = artist_info["genres"]

            #Album
            album = track["track"]["album"]["name"]

            #Popularity of the track
            track_pop = track["track"]["popularity"]

            # We get audio features
            audio_features = sp.audio_features(track_uri)[0]

            # We look for the preview url
            preview_url = track["track"]['preview_url']

            # We get the cover url
            cover_url = track["track"]['album']['images'][0]['url']

            for audio_key in audio_keys:
                dict_data[audio_key] = np.append(dict_data[audio_key], audio_features[audio_key])
            
            # We save the results
            dict_data["artist_name"] = np.append(dict_data["artist_name"], artist_name)
            dict_data["artist_pop"] = np.append(dict_data["artist_pop"], artist_pop)
            dict_data["track_name"] = np.append(dict_data["track_name"], track_name)
            dict_data["track_pop"] = np.append(dict_data["track_pop"], track_pop)
            dict_data["album"] = np.append(dict_data["album"], album)
            
            # Prints
            print(artist_name, "-", track_name)
        except TypeError:
            print("Skipped", artist_name, "-", track_name)
            continue

    try:
        # Convert to dataframe
        df = pd.DataFrame(dict_data)
        df.set_index("artist_name", inplace = True)
    except ValueError:
        df = pd.DataFrame()

    return df

In [8]:
def get_audio_features_from_track_batches(track_batches):
    df = pd.DataFrame()
    for track_batch in track_batches:
        time.sleep(10)
        try:
            df_ = get_audio_features_from_tracks(sp, track_batch)
            df = pd.concat([df,df_], axis=0)
            
        except ReadTimeout:
            print('Spotify timed out... trying again...')

            # We try again
            df_ = get_audio_features_from_tracks(sp, track_batch)
            df = pd.concat([df,df_], axis=0)
            
    return df

# We load the existing data

In [9]:
df = pd.read_csv("data/track_audio_features_updated.csv", sep=";", index_col = [0])
playlists = pd.read_csv("data/playlist_links.csv", sep=";")
playlist_links = np.concatenate(playlists.values)

The idea is that we want to go through the list of playlists and existing tracks and update with any new additions (in a fast way)

In [10]:
# We get all the tracks that are currently in all the playlists
tracks = get_multiple_playlist_tracks(playlist_links)

In [11]:
tracks, track_names = return_unique_tracks_with_data(tracks)

In [12]:
tracks.shape

(5988,)

In [13]:
df.shape

(2866, 12)

We now only want to only extract the selected features for any new tracks and simply add them on top of what we had already 

# Extract features for new tracks

In [14]:
# We get the new tracks by checking if they are already in the downloaded track features
new_tracks = np.array([track for track in tracks if track["track"]["name"] not in df['track_name'].values])

In [15]:
# We specify a batch size to not break the API
batch_size = 50
n = new_tracks.shape
n_batches = np.floor(new_tracks.shape[0] / batch_size)
track_batches = np.array_split(new_tracks,n_batches)

In [16]:
df_updates = get_audio_features_from_track_batches(track_batches)

track 0 out of 50
Paolo Pellegrino - '74 - '75 (feat. Susan Tyler) - Radio Edit
track 1 out of 50
Flux Pavilion - (Never Gonna) Be Alone
track 2 out of 50
offrami - (i am) here
track 3 out of 50
Rain Radio - 079
track 4 out of 50
Zero - 1 More
track 5 out of 50
Black Girl / White Girl - 101010101 (3D)
track 6 out of 50
Afrojack - 1234 (feat. MC Ambush)
track 7 out of 50
Durante - 13 Voices
track 8 out of 50
Corren Cavini - 1635
track 9 out of 50
SICKOTOY - 2 High 2 Care
track 10 out of 50
Model Man - 2005 NYE
track 11 out of 50
Carla Monroe - 22 Love
track 12 out of 50
DVBBS - 24k
track 13 out of 50
Super Flu - 2daBeat
track 14 out of 50
FooR - 3 Words
track 15 out of 50
Chris Lake - 400
track 16 out of 50
Kaskade - 4AM
track 17 out of 50
Aero Chord - 4U
track 18 out of 50
Steff da Campo - 5 On It - 71 Digits Remix
track 19 out of 50
Massane - 5am
track 20 out of 50
DJ Plaze - 909s & Love
track 21 out of 50
Syzz - 90s Better
track 22 out of 50
R3HAB - 911
track 23 out of 50
DJ Snake - 

Wa-Fu - Attitude
track 22 out of 50
Solounge - Audicity
track 23 out of 50
David Scott - Aurora
track 24 out of 50
Inrayzex - Autumn
track 25 out of 50
CJ RcM - Autumn - Original Mix
track 26 out of 50
Ms. Velvet - Avenue
track 27 out of 50
Y-DAPT - Awesome
track 28 out of 50
Jordin Post - Azure
track 29 out of 50
Jordin Post - Azure - Extended Mix
track 30 out of 50
Vins Massaro - B-Lodge - Luzio remix
track 31 out of 50
Vins Massaro - B-Lodge - Original Mix
track 32 out of 50
Disco Lines - BABY GIRL
track 33 out of 50
Garmiani - BARRACA (feat. MC Pikachu)
track 34 out of 50
MadBliss - BIG BOY TIME
track 35 out of 50
Tiësto - BOOM - Mr.Black Remix
track 36 out of 50
Disclosure - BOSS - Radio Edit
track 37 out of 50
ESSEL - BUBBLEGUM
track 38 out of 50
Tokyo Machine - BUBBLES
track 39 out of 50
Disco Lines - Baby Girl
track 40 out of 50
Mesto - Back & Forth
track 41 out of 50
Darius Syrossian - Back In The Dance
track 42 out of 50
Marshmello - Back In Time
track 43 out of 50
Don Diablo

Lstn - Bouquet
track 43 out of 50
Shermanology - Boyz N Da Club
track 44 out of 50
Shermanology - Boyz N Da Club - Radio Edit
track 45 out of 50
Somna - Brave - ReOrder & Jordan Tobias presents Crowd+Ctrl Remix
track 46 out of 50
Martin Garrix - Breach (Walk Alone)
track 47 out of 50
Rhythm On The Loose - Break Of Dawn
track 48 out of 50
Achilles - Break The Rules
track 49 out of 50
Declain - Break The Silence
track 50 out of 50
MaRLo - Break Through The Ground - Radio Edit
track 0 out of 50
Covex - Breakaway
track 1 out of 50
JVNA - Breakdown
track 2 out of 50
Noisestorm - Breakdown VIP
track 3 out of 50
JODA - Breaking Down Walls
track 4 out of 50
Louis The Child - Breaking News (with RAYE)
track 5 out of 50
Alex Sonata & TheRio - Breath Of The Wild
track 6 out of 50
Chusap - Breathe - DJ Fuel Remix
track 7 out of 50
Aly & Fila - Breathe Us To Life - Monoverse Remix
track 8 out of 50
Ben Böhmer - Breathing
track 9 out of 50
Minus One - Breathing - StanV Remix
track 10 out of 50
Hoodb

Giants' Nest - Clouds & Rainbows
track 2 out of 50
Sosa UK - Clutta
track 3 out of 50
MYDIS - Cobra
track 4 out of 50
Satin Jackets - Coffee and Feels
track 5 out of 50
Moe Turk - Coke & Rum
track 6 out of 50
KC Lights - Cold Light
track 7 out of 50
Major Lazer - Cold Water
track 8 out of 50
BLR - Colibrí
track 9 out of 50
GRiZ - Color Of Your Soul
track 10 out of 50
Milky Chance - Colorado - Icarus Remix
track 11 out of 50
Le Youth - Colour
track 12 out of 50
Grum - Come Alive - Tommy Farrow Remix
track 13 out of 50
MPH - Come Back
track 14 out of 50
Eli & Fur - Come Back Around - Eli & Fur Club Mix
track 15 out of 50
Vini Vici - Come Close
track 16 out of 50
Black V Neck - Come On Over
track 17 out of 50
Rudimental - Come Over (feat. Anne-Marie & Tion Wayne)
track 18 out of 50
Zeitgeist - Come Over Here
track 19 out of 50
Nox Vahn - Come Together
track 20 out of 50
Nora En Pure - Come With Me - Radio Mix
track 21 out of 50
W&W - Comin' to Getcha
track 22 out of 50
Dimitri Vangelis & 

Sultan + Shepard - Don't Forget
track 22 out of 49
Diplo - Don't Forget My Love
track 23 out of 49
Blinkie - Don't Give Up (On Love) - Radio Edit
track 24 out of 49
Julie McDermott - Don't Go - Gerd Janson Re-Work - Radio Edit
track 25 out of 49
Julie McDermott - Don't Go - Gerd Janson Re-Work - Shorter Edit
track 26 out of 49
Monolink - Don't Hold Back (Yotto Remix)
track 27 out of 49
Airwolf Paradise - Don't Hurt Me Baby
track 28 out of 49
REESE - Don't Leave Me
track 29 out of 49
Cajama - Don't Let Me Down
track 30 out of 49
Lane 8 - Don't Let Me Go
track 31 out of 49
Armin van Buuren - Don't Let Me Go - Lucas & Steve Remix
track 32 out of 49
Grum - Don't Look Down - Edit
track 33 out of 49
D.T.E - Don't Play by the Rules
track 34 out of 49
PANG! - Don't Speak
track 35 out of 49
Dua Lipa - Don't Start Now - Dom Dolla Remix
track 36 out of 49
Dua Lipa - Don't Start Now - Pink Panda Remix
track 37 out of 49
Belters Only - Don't Stop Just Yet
track 38 out of 49
ACT ON - Don't Stop Movi

...

Tiësto - Show Me - Radio Edit
track 20 out of 49
Beautiful Minds - Show Me How It's Done
track 21 out of 49
Above & Beyond - Show Me Love
track 22 out of 49
Steve Angello - Show Me Love (feat. Robin S.)
track 23 out of 49
Steve Angello - Show Me Love - Jauz Remix
track 24 out of 49
Steve Angello - Show Me Love - Wh0 Remix
track 25 out of 49
Shaun Dean - Shut You Down
track 26 out of 49
Achilles - Sic Mundus
track 27 out of 49
Will Sparks - Sick Like That (feat. Luciana)
track 28 out of 49
Chaya - Sickness
track 29 out of 49
Blinders - Side 2 Side - SWACQ Edit
track 30 out of 49
Bandēs - Sightglass
track 31 out of 49
Rylan Taggart - Signal
track 32 out of 49
Cornelius SA - Silhouette
track 33 out of 49
Meridian - Silver
track 34 out of 49
Bakai - Simply Put
track 35 out of 49
Lange - Sincere - Tomac Remix
track 36 out of 49
Raven & Kreyn - Sing For You
track 37 out of 49
Deepmaniak - Sing It Back
track 38 out of 49
Anderholm - Sinus
track 39 out of 49
Sub Focus - Siren
track 40 out of 4

Alexis Dex - Suede
track 35 out of 49
Qlank - Suffocating
track 36 out of 49
MYLØ - Sufriendo
track 37 out of 49
Pixl - Sugar Rush
track 38 out of 49
Bonsaye - Sumatra
track 39 out of 49
Noizu - Summer 91
track 40 out of 49
Noizu - Summer 91 (Looking Back)
track 41 out of 49
Hardwell - Summer Air (Sunnery James & Ryan Marciano Remix) - Mix Cut
track 42 out of 49
Martin Garrix - Summer Days (feat. Macklemore & Patrick Stump of Fall Out Boy) - Haywyre Remix
track 43 out of 49
Martin Garrix - Summer Days (feat. Macklemore & Patrick Stump of Fall Out Boy) - Tiësto Remix
track 44 out of 49
Jesse Bru - Summer Rain
track 45 out of 49
Co Kluin - Summer Starters
track 46 out of 49
Massane - Summer's End
track 47 out of 49
NoMBe - Summer's Gone (NoMBe & Thutmose vs. Sonny Alven) - Remix
track 48 out of 49
OCULA - Summit
track 49 out of 49
Above & Beyond - Sun & Moon - Blastoyz Remix
track 0 out of 49
Cloonee - Sun Goes Down
track 1 out of 49
Above & Beyond - Sun In Your Eyes - Spencer Brown Edit

ATTLAS - The Night Air Was Cool
track 4 out of 49
Kasper Koman - The Observer
track 5 out of 49
Blaikz - The Ocean
track 6 out of 49
Habstrakt - The One - NGHTMRE Remix
track 7 out of 49
ThomTree - The Ones We Hate the Most
track 8 out of 49
Martin Garrix - The Only Way Is Up
track 9 out of 49
Martin Garrix - The Only Way Is Up - Radio Edit
track 10 out of 49
Christian Cambas - The Outsiders - T78 Remix
track 11 out of 49
LUM!X - The Passenger (LaLaLa) [feat. MOKABY]
track 12 out of 49
BIJOU - The Players Anthem
track 13 out of 49
Party Pupils - The Plug (feat. Drelli)
track 14 out of 49
Timmy Trumpet - The Prayer (feat. Zafrir)
track 15 out of 49
Eli & Fur - The Pressure
track 16 out of 49
Monolink - The Prey
track 17 out of 49
&ME - The Rapture Pt.II
track 18 out of 49
Alex Sonata & TheRio - The Real Thing
track 19 out of 49
Don Diablo - The Rhythm
track 20 out of 49
Dance Fruits Music - The Rhythm Of The Night
track 21 out of 49
Tiësto - The Right Song
track 22 out of 49
Lane 8 - Th

Millennial Matt - Vacay - Tropical Jazz Mix
track 24 out of 49
Sultan + Shepard - Vahé
track 25 out of 49
Slushii - Valhalla
track 26 out of 49
Zomboy - Valley Of Violence
track 27 out of 49
RÜFÜS DU SOL - Valley of the Yuccas - Live from Joshua Tree
track 28 out of 49
Twelwe - Varuna
track 29 out of 49
Excision - Vault
track 30 out of 49
Hot Since 82 - Veins
track 31 out of 49
HamzeH - Venture
track 32 out of 49
ANOTR - Vertigo
track 33 out of 49
GRiZ - Vibe Check
track 34 out of 49
Naoko Omi - Vibe Ticket
track 35 out of 49
AC Slater - Vibes on Tap
track 36 out of 49
Sonny Fodera - Vibrate
track 37 out of 49
Dan Stone - Vice Versa - John Grand Remix
track 38 out of 49
Will Sparks - Vienna
track 39 out of 49
Louie Vega - Vince Montana Tribute - LV & Rick Wilhite ReWork
track 40 out of 49
Le Youth - Virgo
track 41 out of 49
Alan Morris - Viridian
track 42 out of 49
Grum - Virtual Companion - Edit
track 43 out of 49
Lane 8 - Visions
track 44 out of 49
Lane 8 - Visions - Ocula Remix
tra

Duke Dumont - Won't Look Back
track 39 out of 49
R3HAB - Won't Stop Rocking
track 40 out of 49
Avoure - Wonder
track 41 out of 49
Linier - Wonder - Radio Edit
track 42 out of 49
Skipped The Waked - Wonder Where You Are
track 43 out of 49
Yotto - Wondering
track 44 out of 49
Nu Aspect - Words
track 45 out of 49
ZOOTAH - Work
track 46 out of 49
Dr. Fresch - Work (feat. Chetta)
track 47 out of 49
Bob Sinclar - World Hold On - Fisher Rework
track 48 out of 49
Bob Sinclair - World, Hold On
track 49 out of 49
Ferry Tayle - XII (TrancefamilySF Anthem)
track 0 out of 49
Crisp Nibblets - Xasnas
track 1 out of 49
FISHER - Ya Didn't
track 2 out of 49
BAYNK - Years
track 3 out of 49
OCULA - Years On End
track 4 out of 49
LeeDee - Yeesh Euphemism
track 5 out of 49
Fabio Aurea - Yini - Serge Devant Remix
track 6 out of 49
Alex M.O.R.P.H. - You
track 7 out of 49
JKAY - You & I
track 8 out of 49
Sonny Fodera - You & I (feat. Janai)
track 9 out of 49
Black Caviar - You Ain't Gotta Bounce to This (Lean)

# Add to existing features

In [17]:
df_updated = pd.concat([df, df_updates], axis = 0)

In [18]:
df_updated.shape

(6023, 12)

In [19]:
df_updated.to_csv("data/track_audio_features_updated.csv", sep = ";", index = [0])